# 实习五 温湿特征量和部分稳定度指数的计算
## 1. 位温和相当位温（起始高度按1000hPa来进行计算）
## 2. （2）绘制从7月17日00时-7月22日18时，沿35°N， 135°E的位温，相当位温时间-高度剖面图，并分析是否存在不稳定区域。
## 3. 尝试计算位势稳定度指数Eσ和K指数并根据第六章所学内容进行分析。计算公式如下
### 2021/11/27

In [ ]:
########需要用到的模块
import os
import numpy as np  # 调用numpy
import pandas as pd
import xarray as xr
from math import *
import metpy.calc as mpcalc
from metpy.units import units
import metpy.constants as constants
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [ ]:
plt.rcParams['font.sans-serif'] = ['SimHei']  ###防止无法显示中文并设置黑体
plt.rcParams['axes.unicode_minus'] = False  ###用来正常显示负号

In [ ]:
##时间处理,加个八小时
def shijianchuli(img_name):
    bigmonth = ['01', '03', '04', '05', '07', '08', '10', '12']
    img_name_year = img_name[:4]
    img_name_month = img_name[4:6]
    img_name_day = img_name[6:8]
    img_name_hour = str(int(img_name[8:10]) + 108)  ###转换成北京时间，并且为了小时显示00这样的格式，加了108，后面再截取

    ####对日期的处理，有些加了八小时变成北京时间之后，日期会发生改变，下面就是对日期在闰年、非闰年，大小月等情况时的处理
    if int(img_name_hour) > 124:
        img_name_day = str(int(img_name_day) + 101)
        img_name_day = img_name_day[1:3]
        img_name_hour = str(int(img_name_hour) - 24)
        img_name_hour = img_name_hour[1:3]
        if int(img_name_year) % 4 == 0 and int(img_name_year) % 100 != 0:  ####闰年的判定
            if img_name_month == '02':
                if int(img_name_day) > 29:
                    img_name_day = '01'
            if img_name_month in bigmonth:
                if int(img_name_day) > 31:
                    img_name_month = str(int(img_name_month) + 1)
                    img_name_day = '01'
            else:
                if int(img_name_day) > 30:
                    img_name_month = str(int(img_name_month) + 1)
                    img_name_day = '01'
        else:
            if img_name_month == '02':
                if int(img_name_day) > 28:
                    img_name_day = '01'
            if img_name_month in bigmonth:
                if int(img_name_day) > 31:
                    img_name_month = str(int(img_name_month) + 1)
                    img_name_day = '01'
            else:
                if int(img_name_day) > 30:
                    img_name_month = str(int(img_name_month) + 1)
                    img_name_day = '01'
    else:
        img_name_hour = img_name_hour[1:3]
    daytime = img_name_year + '_' + img_name_month + '_' + img_name_day + '_' + img_name_hour
    return daytime

In [ ]:
all = xr.open_dataset(r'D:\python\tianzhen\shixi3_4\ds_hgt_t_uv.nc')
rh_all = xr.open_dataset(r'D:\python\tianzhen\shixi3_4\ds_rh.nc')

In [ ]:
all

In [ ]:
rh_all

In [ ]:
hgt = all['hgt'][:, :, :, :]
Temp = all['t'][:, :, :, :]
rh = rh_all['rh'][:, :, :, :]
lon = all['lon'][:]
lat = all['lat'][:]
time = all['time'][:]
level = rh_all['level'][:]
level1 = all['level'][:]

In [ ]:
hgt=hgt*units.dagpm
Temp=Temp*units.degC

In [ ]:
time.data

In [ ]:
P=level1*units.hPa

In [ ]:
weiwen = np.zeros((len(level1), len(time), len(lat), len(lon)))
for h_i in range(len(level1)):
    weiwen= mpcalc.potential_temperature(P,Temp)

In [ ]:
weiwen